In [ ]:
%%capture
!git clone -q https://github.com/comfyanonymous/ComfyUI
%cd /content/ComfyUI
!pip install -q -r requirements.txt
!apt -y install -qq aria2

!aria2c -q -c -x 16 -s 16 -k 1M https://huggingface.co/T5B/Z-Image-Turbo-FP8/resolve/main/z-image-turbo-fp8-e4m3fn.safetensors -d models/diffusion_models -o z-image-turbo-fp8-e4m3fn.safetensors
!aria2c -q -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/text_encoders/qwen_3_4b.safetensors -d models/clip -o qwen_3_4b.safetensors
!aria2c -q -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/vae/ae.safetensors -d models/vae -o ae.safetensors

import os, random, torch, math
import numpy as np
from PIL import Image
from nodes import NODE_CLASS_MAPPINGS

UNETLoader = NODE_CLASS_MAPPINGS['UNETLoader']()
CLIPLoader = NODE_CLASS_MAPPINGS['CLIPLoader']()
VAELoader = NODE_CLASS_MAPPINGS['VAELoader']()
CLIPTextEncode = NODE_CLASS_MAPPINGS['CLIPTextEncode']()
KSampler = NODE_CLASS_MAPPINGS['KSampler']()
VAEDecode = NODE_CLASS_MAPPINGS['VAEDecode']()
EmptyLatentImage = NODE_CLASS_MAPPINGS['EmptyLatentImage']()

with torch.inference_mode():
    unet = UNETLoader.load_unet('z-image-turbo-fp8-e4m3fn.safetensors','fp8_e4m3fn_fast')[0]
    clip = CLIPLoader.load_clip('qwen_3_4b.safetensors', type='lumina2')[0]
    vae = VAELoader.load_vae('ae.safetensors')[0]

@torch.inference_mode()
def generate(prompt, negative, w, h, steps, cfg, seed, sampler, scheduler, denoise, batch):
    out='/content/ComfyUI/output'
    os.makedirs(out, exist_ok=True)
    if seed==0:
        seed=random.randint(0,2**64-1)
    pos=CLIPTextEncode.encode(clip,prompt)[0]
    neg=CLIPTextEncode.encode(clip,negative)[0]
    latent=EmptyLatentImage.generate(w,h,batch_size=batch)[0]
    samp=KSampler.sample(unet,seed,steps,cfg,sampler,scheduler,pos,neg,latent,denoise=denoise)[0]
    dec=VAEDecode.decode(vae,samp)[0]
    paths=[]
    for i in range(batch):
        img=Image.fromarray((dec[i].cpu().numpy()*255).astype('uint8'))
        p=f"{out}/zimg_{seed}_{i}.png"
        img.save(p)
        paths.append(p)
    return paths, seed

def make_grid(imgs, cols):
    rows=math.ceil(len(imgs)/cols)
    w,h=imgs[0].size
    g=Image.new('RGB',(cols*w,rows*h))
    for i,im in enumerate(imgs):
        g.paste(im,((i%cols)*w,(i//cols)*h))
    return g

import ipywidgets as w
from IPython.display import display, clear_output

PRESETS={
 'Cinematic':'cinematic lighting, ultra realistic, shallow depth of field, film still',
 'Portrait':'studio portrait, soft light, high detail skin texture',
 'Anime':'anime style, clean lineart, vibrant colors',
 'Product':'product photography, studio lighting, sharp focus, white background',
 'NSFW Soft':'sensual pose, aesthetic nude art, soft lighting, artistic photography'
}

BASE_NEG='blurry, low quality, bad anatomy, deformed'
NSFW_NEG='explicit sexual acts, porn, extreme fetish'

preset=w.Dropdown(options=list(PRESETS.keys()),description='Preset')
nsfw=w.Checkbox(False,description='NSFW')

prompt=w.Textarea(layout=w.Layout(width='650px',height='90px'))
neg=w.Textarea(layout=w.Layout(width='650px',height='60px'))

width=w.IntSlider(1024,512,2048,64,description='Width')
height=w.IntSlider(1024,512,2048,64,description='Height')
steps=w.IntSlider(9,1,50,description='Steps')
cfg=w.FloatSlider(1,0.1,10,0.1,description='CFG')
denoise=w.FloatSlider(1,0.1,1,0.05,description='Denoise')
batch=w.IntSlider(1,1,4,description='Batch')
seed=w.IntText(0,description='Seed')

sampler=w.Dropdown(options=['euler','euler_ancestral','dpm_fast'],description='Sampler')
scheduler=w.Dropdown(options=['simple','karras'],description='Scheduler')

grid_on=w.Checkbox(False,description='Grid')
cols=w.IntSlider(2,2,4,description='Cols')

btn=w.Button(description='GENERATE',button_style='success')
out=w.Output()

def run(b):
    with out:
        clear_output(wait=True)
        p=f"{PRESETS[preset.value]}, {prompt.value}"
        n=BASE_NEG
        if not nsfw.value:
            n+=', '+NSFW_NEG
        paths,sd=generate(p,n,width.value,height.value,steps.value,cfg.value,seed.value,sampler.value,scheduler.value,denoise.value,batch.value)
        imgs=[Image.open(i) for i in paths]
        print('Seed:',sd)
        if grid_on.value and len(imgs)>1:
            display(make_grid(imgs,cols.value))
        else:
            for i in imgs: display(i)

btn.on_click(run)

display(w.VBox([preset,nsfw,prompt,neg,width,height,steps,cfg,denoise,batch,seed,sampler,scheduler,grid_on,cols,btn,out]))